#### merge peak by MSPC

In [4]:
####### normalize
######## based on pvalue
os.chdir("/Charles/mjy/210505_A00869_0471_BH3NHMDSX2/peak_calling/merged_peak_MSPC/")
macs_result_dir = "normalized_macs"
metpeak_result_dir = "normalized_metpeak"
os.system("mkdir -p %s" % macs_result_dir)
os.system("mkdir -p %s" % metpeak_result_dir)

def normalize_bed(bed, prefix, result_dir):
    normalized_script = "/home/software/m6APipe/m6APipe/bin/normalize_peaks.py"
    result_bed = os.path.join(result_dir, "%s.bed"%prefix)
    os.system("python %s %s %s" % (normalized_script, bed, result_bed))
    
###
macs_dir = "/Charles/mjy/210505_A00869_0471_BH3NHMDSX2/peak_calling/macs2/"
macs_list = glob.glob("%s/*_normalized.bed" % macs_dir)  # macs2_mus_IP_5_normalized.bed
for macs in macs_list:
    prefix = os.path.basename(macs).split("macs2_")[1].split("_normalized.bed")[0]
    normalize_bed(macs, prefix, macs_result_dir)
###
metpeak_dir = "/Charles/mjy/210505_A00869_0471_BH3NHMDSX2/peak_calling/MeTPeak/"
metpeak_list = glob.glob("%s/*/*.bed6" % metpeak_dir)
for metpeak in metpeak_list:
    prefix = metpeak.split("/")[-2]
    normalize_bed(metpeak, prefix, metpeak_result_dir)

In [5]:
#### based on pvalue (MSPC method)
##############################################
os.chdir("/Charles/mjy/210505_A00869_0471_BH3NHMDSX2/peak_calling/merged_peak_MSPC/")
##############################################
metpeak_dir = "normalized_metpeak/"
metpeak_list = glob.glob("%s/*.bed" % metpeak_dir) # CRR055555.bed
###
macs_dir = "normalized_macs/"
macs_list = glob.glob("%s/*.bed" % macs_dir)  # CRR055555.bed
###
for metpeak in metpeak_list:
    sample = os.path.basename(metpeak).split(".bed")[0]
    sub_dir = sample
    os.system("mkdir -p %s" % sub_dir)
    macs = os.path.join(macs_dir, os.path.basename(metpeak))
    os.system("dotnet /home/software/MSPC/CLI.dll -i %s -i %s -r tec -c 2 -w 1e-2 -s 1e-4 -o %s"%(metpeak,macs,sub_dir))

#### Merge peak based on pvalue

In [14]:
# ####################################################
# os.chdir("/Charles/mjy/210505_A00869_0471_BH3NHMDSX2/peak_calling/merged_peak_MSPC/")
# bed_list = glob.glob("*/ConsensusPeaks.bed")
# print(len(bed_list))
# #####################################################
# # result_dir = "union_peak_bedtools/"
# # os.system("mkdir -p %s" % result_dir)
# # result_bed = "merge_peak2.bed"
# # os.system("cp %s %s" % (bed_list[0], result_file))


# def merge_twoSamples(bed_1, bed_2):
#     print(bed_1, bed_2)
#     # result_tmp = "%s_%s.bed" % (bed_1.split("/")[-2], bed_2.split("/")[-2])
#     result_tmp = "any.bed"
#     ###
#     tmp_1 = "tmp_1.bed"
#     os.system("bedtools intersect -a %s -b %s -v -wa > %s" % (bed_1, bed_2, tmp_1))
#     df_1 = pd.read_table(tmp_1, sep="\t", header=None)
#     ###
#     tmp_2 = "tmp_2.bed"
#     os.system("bedtools intersect -a %s -b %s -v -wa > %s" % (bed_2, bed_1, tmp_2))
#     df_2 = pd.read_table(tmp_2, sep="\t", header=None)
#     ###
#     tmp_3 = "tmp_3.bed"
#     os.system("bedtools intersect -a %s -b %s -wa -wb > %s" % (bed_1, bed_2, tmp_3))
#     df = pd.read_table(tmp_3, sep="\t", header=None)
#     df_a = df[df.iloc[:,4] >= df.iloc[:,9]].dropna()
#     df_a = df_a.iloc[:,0:5]
#     df_b = df[df.iloc[:,4] < df.iloc[:,9]].dropna()
#     df_b = df_a.iloc[:,5:10]
#     df_3 = pd.concat([df_a, df_b])
#     ### combine
#     df_res = pd.concat([df_1, df_2, df_3]).dropna()
# #     print(df_res)
#     df_res.iloc[:,1] = df_res.iloc[:,1].astype(int)
#     df_res.iloc[:,2] = df_res.iloc[:,2].astype(int)
#     df_res.to_csv(result_tmp, sep="\t", header=False, index=False)
#     os.remove(tmp_1)
#     os.remove(tmp_2)
#     os.remove(tmp_3)
#     return result_tmp

# result_bed = bed_list[0]
# for bed in bed_list[1:]:
#     result_bed = merge_twoSamples(bed, result_bed)

5
NC/ConsensusPeaks.bed NC2/ConsensusPeaks.bed
T2/ConsensusPeaks.bed any.bed
T1/ConsensusPeaks.bed any.bed
T3/ConsensusPeaks.bed any.bed


#### merge peak by bedtools

In [9]:
#####################################################
os.chdir("/Charles/project/ASm6A/peak_calling/")
macs_dir = "merged_peak_MSPC/normalized_macs/"
metpeak_dir = "merged_peak_MSPC/normalized_metpeak/"
#####################################################
result_dir = "merged_peak_bedtools/"
os.system("mkdir -p %s" % result_dir)

metpeak_list = glob.glob("%s/*.bed" % metpeak_dir)
for metpeak in metpeak_list:
    sample = os.path.basename(metpeak).split(".bed")[0]
    sub_dir = os.path.join(result_dir, sample)
    os.system("mkdir -p %s" % sub_dir)
    macs = os.path.join(macs_dir, os.path.basename(metpeak))
    res = os.path.join(sub_dir, "ConsensusPeaks.bed")
    comm = "bedtools intersect -a %s -b %s -wa | sort -k1,1 -k2,2n | uniq > %s" % (macs, metpeak, res)
#     print(macs)
#     print(metpeak)
    os.system(comm)

#### union peaks 

In [3]:
####################################################
os.chdir("/Charles/project/ASm6A/peak_calling/")
macs_dir = "merged_peak_MSPC/normalized_macs/"
metpeak_dir = "merged_peak_MSPC/normalized_metpeak/"
#####################################################
result_dir = "union_peak_bedtools/"
os.system("mkdir -p %s" % result_dir)

metpeak_list = glob.glob("%s/*.bed" % metpeak_dir)
for metpeak in metpeak_list:
    sample = os.path.basename(metpeak).split(".bed")[0]
    sub_dir = os.path.join(result_dir, sample)
    os.system("mkdir -p %s" % sub_dir)
    macs = os.path.join(macs_dir, os.path.basename(metpeak))
    res = os.path.join(sub_dir, "ConsensusPeaks.bed")
    comm = "cat %s %s | sort -k1,1 -k2,2n | bedtools merge -i - > %s" % (macs, metpeak, res)
    os.system(comm)

#### union three methods (macs2 + MeTPeak + meyer)

In [5]:
####################################################
os.chdir("/Charles/project/ASm6A/peak_calling/")
macs_dir = "merged_peak_MSPC/normalized_macs/"
metpeak_dir = "merged_peak_MSPC/normalized_metpeak/"
meyer_dir = "/Charles/project/ASm6A/peak_calling/meyer/"
#####################################################
result_dir = "union_three_bedtools/"
os.system("mkdir -p %s" % result_dir)

metpeak_list = glob.glob("%s/*.bed" % metpeak_dir)
for metpeak in metpeak_list:
    sample = os.path.basename(metpeak).split(".bed")[0]
    sub_dir = os.path.join(result_dir, sample)
    os.system("mkdir -p %s" % sub_dir)
    macs = os.path.join(macs_dir, os.path.basename(metpeak))
    meyer = os.path.join(meyer_dir, "meyer_%s_normalized.bed"%sample)  # meyer_SRR8209867_normalized.bed
    res = os.path.join(sub_dir, "ConsensusPeaks.bed")
    comm = "cat %s %s %s | sort -k1,1 -k2,2n | bedtools merge -i - > %s" % (macs, metpeak, meyer, res)
    os.system(comm)

In [8]:
### 
# peak_dir = "/home/galaxy/project/alleleSpecific_analysis/data/m6a_peak/peak_calling/bam/combined_peak/rename/"
peak_dir = "/home/galaxy/project/alleleSpecific_analysis/data/m6a_peak/peak_calling/bam/macs2/peak/rename/"
bed_list = glob.glob("%s/*.bed" % peak_dir)
asm6a_dir = "/home/galaxy/project/alleleSpecific_analysis/results/ASm6A_addZr/"
for bed in bed_list:
    sample = os.path.basename(bed)
    asm6a = os.path.join(asm6a_dir, sample)
    if os.path.exists(bed) and os.path.exists(asm6a):
        # res = os.path.join(result_dir, "%s.bed"%sample)
        asm6a_num = int(os.popen("wc -l %s"%asm6a).read().split()[0])
        com = "bedtools intersect -a %s -b %s -wa | sort -u | wc -l" % (asm6a, bed)
        within_num = int(os.popen(com).read().split()[0])
        print(os.path.basename(bed), within_num, within_num/asm6a_num)
    else:
        print("absent: %s" % bed)

Lung-2-1.bed 156 0.26621160409556316
FrontalCortex_3.bed 606 0.09728688393000481
Lung-2-4.bed 357 0.2489539748953975
Heart_1.bed 378 0.11440677966101695
Cerebellum_3.bed 996 0.11295078249036063
brain_IP_1.bed 2472 0.7613181398213735
Muscle_1.bed 179 0.09123343527013252
Kidney_3.bed 556 0.16134648868253046
Jejunum-5-3.bed 290 0.3988995873452545
Esophagus-4-2.bed 516 0.37719298245614036
Lung_3.bed 137 0.06618357487922705
Adipose-1-1.bed 1699 0.3863119599818099
Muscle-5-3.bed 602 0.36177884615384615
Appendix-3-2.bed 356 0.34396135265700484
heart_IP_1.bed 870 0.8146067415730337
Urinary-bladder-4-2.bed 1379 0.3791586472367336
heart_IP_3.bed 1423 0.8239722061378112
Lung_1.bed 349 0.13469702817445
Rectum-5-3.bed 187 0.4221218961625282
Duodenum-5-3.bed 306 0.3853904282115869
kidney_IP_3.bed 1821 0.8000878734622144
lung_IP_5.bed 1064 0.918825561312608
FrontalCortex_2.bed 1332 0.1385623634661396
Muscle_3.bed 433 0.13121212121212122
Urinary-bladder-5-3.bed 282 0.30257510729613735
Cerebrum-5-3.bed